# Compute diffusion scores in Python

In [1]:
import networkx as nx
import numpy as np
import logging
import scipy as sp
from math import pi, sqrt
import sys
import os 

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

dir_path = os.path.dirname(os.path.realpath('__file__'))

DEBUG:root:test


#### Import kernel functions from diffuPy

The kernel functions a imported from the package. Despite this the functions implementation are in this notebook (final _imp in the function name).

In [2]:
from diffupy.kernel import commute_time_kernel, p_step_kernel, inverse_cosine_kernel, diffusion_kernel, regularised_laplacian_kernel

from diffupy.matrix import Matrix, LaplacianMatrix

from diffupy.miscellaneous import get_label_list_graph

### Import example graph

In [3]:
G = nx.read_gml(dir_path+'/04_unit_testing/_graph.gml', label='id')

### General functions

#### Labels mapping

In [4]:
def csv_labeled_matrix_to_matrix(path):
    # Import matrix from csv file and remove headers
    m = np.genfromtxt(path, dtype=None, delimiter=',')
    return Matrix(np.array([[float(x) for x in a[1:]] for a in m[1:]]),os.path.basename(path).split('.csv'), m[1:,0], m[0, 1:])                                                                                              
                                                                                                                    

In [5]:
def run_score_test(score_func, G, input_scores, test_output_scores, z = False):
    
    computed_output_scores = score_func(G, input_scores, z)

    if isinstance(computed_output_scores, Matrix):
        computed_output_scores = computed_output_scores.mat
    
    if isinstance(test_output_scores, Matrix):
        test_output_scores = test_output_scores.mat
    
    logging.info(' %s  \n %s\n', 'Computed matrix', computed_output_scores)
    logging.info(' %s  \n %s\n', 'Test matrix', test_output_scores)
    # Assert rounded similarity (floating comma)
    assert np.allclose(computed_output_scores, test_output_scores)
    logging.info(' Test '+ score_func.__name__ +' passed')

#### Helpers

In [6]:
# In which format is the input? Tell apart vector, matrix or list of matrices
def which_format(x):
    # if is.numeric(x) and is.null(dim(x)): return "vector"
    if isinstance(x, isinstance(x[0], list)) or isinstance(x, isinstance(x[0], np.ndarray)): return "matrix"
    if isinstance(x, list): return "list"
    
    raise ValueError('Non-recognised input scores format, object of class:  %s', x.__class__.__name__)

#### Checks

In [7]:
# TODO
#.check_method
#.check_metric

# Check scores sanity
def check_scores(scores):
    # TODO
    #if np.equal(scores.cols_labels, any(([], None, 'Nan')):
    if scores.cols_labels == []:
        raise ValueError("Scores must be a named list but supplied list contains no names.")

    if not 'float' and 'int' in str(scores.mat.dtype):
        raise ValueError("The scores in background are not numeric.")
        
    for score, col_label, row_label in iter(scores):
        if not isinstance(score, float) and not isinstance(score, int):
            raise ValueError("The scores in background are not numeric")
        if score in ['Nan', None]:
            raise ValueError("Scores input cannot contain NA. But background .")
        if col_label in ['Nan', None]:
            raise ValueError("The scores in background must have rownames according to the scored nodes.")
        if row_label in ['Nan', None]:
            raise ValueError("The scores in background must have colnames to differentiate score sets.")
            
    std_mat = Matrix(np.std(scores.mat, axis=0), ['sd'], scores.cols_labels)

    for sd, col_label, row_label in iter(std_mat):
        if sd in ['Nan', None]:
            raise ValueError("Standard deviation in background is NA in column:" + str(col_label))
        if sd == 0:
            raise ValueError("Standard deviation in background is 0 in column:" + str(col_label))

In [8]:
# Check graph sanity
def check_graph(graph):
    if graph in [None, 'NA', 'Nan']:
        raise ValueError("'graph' missing")

    if not isinstance(graph, nx.Graph):
        raise ValueError("'graph' must be an NetworkX graph object")

    nodes_names = get_label_list_graph(graph, 'name')
    if nodes_names in [None, 'NA', 'Nan']:
        raise ValueError("'graph' must have node names.")

    if any(nodes_names) is None:
        raise ValueError("'graph' cannot have NA as node names")

    if len(np.unique(nodes_names)) != len(nodes_names):
        raise ValueError("'graph' has non-unique names! Please check that the names are unique.")
                         
    if nx.is_directed(graph):
        raise Warning("graph' should be an undirected NetworkX graph object.")

    edge_weights = nx.get_edge_attributes(graph,'weight')
    if edge_weights:
        if any(edge_weights) is None:
            raise ValueError("'graph' cannot contain NA edge weights, all must have weights.")
        if any(edge_weight) < 0:
            raise Warning("'graph' should not contain negative edge weights.")


## Diffuse scores

Diffuse scores on a network Function diffuse takes a network in networkx format and an initial state to score all the nodes in the network.

In [9]:
def calculate_scores(col_ind, scores, diff, const_mean, const_var):
    col_in = scores[:, col_ind]
    col_raw = diff[:, col_ind]

    s1 = np.sum(col_in)
    s2 = np.sum(col_in**2)

    # means and vars depend on first and second moments
    # of the input. This should be valid for non-binary
    # inputs as well
    score_means = const_mean*s1
    score_vars = const_var*(len(scores)*s2 - s1**2)
    
    return np.subtract(col_raw, score_means)/np.sqrt(score_vars)

def diffuse_raw (graph,
    scores,
    z = False,
    K = None,
    *argv):
    
    # TODO
    # sanity checks
    check_scores(scores)

    # Kernel matrix
    if K is None:
        check_graph(graph)
        logging.info('Kernel not supplied. Computing regularised Laplacian kernel ...')
        K = regularised_laplacian_kernel(graph, normalized = False)
        logging.info('Done')
    else:
        # .check_K(K)
        logging.info('Using supplied kernel matrix...')

    # Compute scores        

    # TODO: match indices, for now we assume indices match
    # for i, input_scores in enumerate(scores):
    #    for j, score in enumerate(input_scores):
    #       id_label_mapping(i)
    # input scores

    # TODO: Sparse
    # scores.mat <- methods::as(scores[[scores.name]], "sparseMatrix")

    n = len(scores.mat)
    
    K = K.mat
    
    # raw scores
    diff = np.matmul(K[:,:n], scores.mat)

    # Return base matrix if it is raw
    # Continue if we want z-scores
    if not z: 
        return diff

    # If we want z-scores, must compute rowmeans and rowmeans2
    row_sums = np.array([round(np.sum(row), 2) for row in K[:, :n]])
    row_sums_2 = np.array([np.sum(row) for row in K[:, :n]**2])

    # Constant terms over columns
    const_mean = row_sums/n
    const_var = np.subtract(n*row_sums_2, row_sums**2)/((n - 1)*(n**2))    
    
    return Matrix(np.transpose([np.array(calculate_scores(i, scores.mat, diff, const_mean, const_var)) for i in range(len(diff[0]))]), scores.rows_labels, scores.cols_labels) 

### Raw scores test

In [10]:
run_score_test(diffuse_raw, G, Matrix.from_csv(dir_path+'/scores_test/input_scores.csv'), Matrix.from_csv(dir_path+'/scores_test/output_scores.csv'))


/Users/jmarinllao/Documents/CREB/diffupy/src/diffupy/matrix.py:141: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  m = np.genfromtxt(path, dtype=None, delimiter=',')
/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
INFO:root:Kernel not supplied. Computing regularised Laplacian kernel ...
INFO:root:Done
INFO:root: Computed matrix  
 [[0.04214556 0.50958041]
 [0.02803012 0.54215142]
 [0.07392846 0.44411547]
 [0.02882746 0.45956469]
 [0.02498394 0.43278159]
 [0.02694918 0.39459411]
 [0.04164558 0.4860914 ]
 [0.02889637 0.40307055]
 [0.03051561 0.4358803 ]
 [0.02342762 0.57536671]
 [0.02099228 0.52941183]
 [0.02743536 0.43368413]
 [0.03170256 0.43409448]
 [0.02845791 0.38471574]
 [0.02020737 0.30761156

### z-scores test

In [11]:
run_score_test(diffuse_raw, G, Matrix.from_csv(dir_path+'/scores_test/input_scores.csv'), Matrix.from_csv(dir_path+'/scores_test/output_z_scores.csv'), z = True)

/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
INFO:root:Kernel not supplied. Computing regularised Laplacian kernel ...
INFO:root:Done
INFO:root: Computed matrix  
 [[ 5.72900503e-01  8.00464965e-01]
 [-1.13346069e-01  1.61790410e+00]
 [ 4.08043369e+00 -5.05016326e-01]
 [-9.23887165e-02 -1.17397845e-02]
 [-4.09220982e-01 -7.60025903e-01]
 [-2.51410271e-01 -1.84482009e+00]
 [ 1.11370470e+00  8.54037530e-01]
 [-5.88803789e-02 -1.03957684e+00]
 [ 3.71221934e-02 -5.94365716e-01]
 [-2.57795602e-01  1.54883188e+00]
 [-4.84654538e-01  1.27826704e+00]
 [-1.25700004e-01 -4.41466521e-01]
 [ 1.33054511e-01 -6.92932980e-01]
 [-7.37208835e-02 -1.23184462e+00]
 [-2.17625149e-01 -1.15913118e+00]
 [-6.20089510e-02 -4.70118127e-01]
 [-2.22217600e-01  6.33062809e-01]
 [-3.76246194e-01  1.23893618e+00]
 [-3.13109721e-01 -1.9